# Leavitt experiment using LLMs as nodes

### Setup

In [1]:
from __future__ import annotations   # needed for Python < 3.14 for type hint of Node class

import networkx as nx
from dataclasses import dataclass, field
from typing import Dict, List, Set, Tuple, Optional, Literal, Type
import matplotlib.pyplot as plt
import random

from openai import OpenAI

from dotenv import load_dotenv
_ = load_dotenv()

#### Define networks and allowed nodes

In [3]:
NODES = ('A','B','C', 'D', 'E')

SYMBOLS: List[str] = [  '\u25A1', # square
                        '\u25B7', # right triangle
                        '\u25C1', # left triangle
                        '\u25C7', # diamond
                        '\u25CB', # circle
                        '\u2606', # star     
                     ]

NETWORKS: Dict[str, Dict[str, List[str]]] = {
    "chain": {
        "A": ["B"],
        "B": ["A", "C"],
        "C": ["B", "D"],
        "D": ["C", "E"],
        "E": ["D"],
    },
    "circle": {
        "A": ["B", "E"],
        "B": ["A", "C"],
        "C": ["B", "D"],
        "D": ["C", "E"],
        "E": ["D", "A"],
    },
    "wheel": {  
        "A": ["C"],
        "B": ["C"],
        "C": ["A", "B", "D", "E"],
        "D": ["C"],
        "E": ["C"],
    },
    "Y": {  
        "A": ["C"],
        "B": ["C"],
        "C": ["A", "B", "D"],
        "D": ["C", "E"],
        "E": ["D"],
    },
}

##### Function to create trial data

* Using the original symbol patterns with one common symbol across all participants

In [4]:
def generate_trial(symbols: List[str],
                   nodes: List[str]):
    '''
    Generate trial data for N nodes from a list of S>N (usually N+1) symbols.
    Constraints: only one symbol from list occurs in ALL node data

    Returns:
        common_symbol: str
        trial_data: Dict[str, List[str]]
    '''

    common_symbol = random.choice(symbols)
    others = [sym for sym in symbols if sym!=common_symbol]
    random.shuffle(others)
    trial_data = {}
    for node, missing in zip(nodes, others):
        trial_data[node] = [sym for sym in symbols if sym!=missing]
        random.shuffle(trial_data[node])
    
    return common_symbol, trial_data



In [5]:
generate_trial(SYMBOLS, NODES)

('☆',
 {'A': ['◇', '☆', '◁', '□', '○'],
  'B': ['☆', '□', '▷', '◁', '◇'],
  'C': ['☆', '○', '◁', '▷', '□'],
  'D': ['□', '☆', '○', '▷', '◇'],
  'E': ['○', '◇', '☆', '◁', '▷']})

### Prompts

In [6]:

SYSTEM_PROMPT =  """
        You are one participant (node) in a 5-person communication network.
        Each of you has been given a card with 5 symbols on it.
        You are trying to find the SINGLE symbol that appears on all 5 cards.
        You ONLY know your own card symbols and messages you receive.
        You may send written messages ONLY to your listed neighbors.
        
        Rules:
        - Do not invent symbols you haven't seen.
        - Prefer sending compact, useful info (e.g., your card or your current candidate set).
        - When you are confident the common symbol is uniquely determined, set current_guess.
        - Output MUST be valid JSON with keys: current_guess, outgoing_messages, notes.
        - outgoing_messages is a list of {to, content} where to is a neighbor.
        """

TURN_PROMPT = """
        You current knowledge is {current_knowledge}

        You can ONLY send messages to NODES {neighbor_str}

        What do you want to do now?
        """

#### Node class 

In [24]:
class Node:

    def __init__(self, 
                 id: str, 
                 network: str,
                 card: str
                ):

        if id not in NODES:
            raise Exception(f'Node id must be one of {NODES}')
        
        self.id = id

        if not network in NETWORKS.keys():
            raise Exception(f'Network must be one of: {NETWORKS.keys()}')
        
        self.network = network

        self.neighbors = NETWORKS[network][id]

        self.initial_data = card
        self.inbox = []
        self.outbox = []

    def __repr__(self):
        return f"""Node(id='{self.id}', network={self.network}, 
                   neighbors={self.neighbors},
                   initial_data={self.initial_data})
                   inbox={self.inbox},
                   outbox={self.outbox}
                   """


    def send_message(self, recipient: Node, content: str):
        """
        Send a message `content` to a neighbor checking it is allowed
        Returns:
            status
        """

        recipient_id = recipient.id

        if recipient_id not in self.neighbors:
            raise Exception(f'Trying to send a message to an non-neighbor')

        try:
            recipient.receive_message(self.id, content)
            self.outbox.append({'to': recipient_id,
                                'message': content})
            return f"Message '{content[:10]}...' sent to Node {recipient_id}'"
        except Exception as e:
            print(e)
            return False


    def receive_message(self, sender: str, content: str):
        """
        Receive a message from a sender and add to inbox
        """

        self.inbox.append({'from': sender, 'message': content})


    def take_turn(self):
        """
        Evaluate current knowledge and decide what action to take
        - send message to a neighbor
        - make a guess
        """

        current_inbox = '\n'.join([ f"{message['from']} said {message['message']}" for message in self.inbox]) if self.inbox else "No interactions"
        
        current_knowledge = f"""
        Your initial data is: { self.initial_data }
        You have learned the following from your interactions:
        {current_inbox}
        """

        messages = [
            {'role': 'system',
             'content': SYSTEM_PROMPT },

            {'role': 'user',
             'content': TURN_PROMPT.format(current_knowledge=current_knowledge, 
                                           neighbor_str = ' and '.join(self.neighbors)) },
        ]

        # TODO - give each node instance a specific LLM
        response = LLM.responses.create(
            model="chatgpt-4o-latest",
            input=messages
        )

        print(response.output_text)

### Testing

In [15]:
LLM = OpenAI()

In [31]:
network_type = 'chain'

common_symbol, trial_data = generate_trial(SYMBOLS, NODES)

nodes = {node: Node(id=node, 
              network=network_type, 
              card = ' '.join(trial_data[node]))
              for node in NETWORKS[network_type]
        }

In [32]:
nodes

{'A': Node(id='A', network=chain, 
                    neighbors=['B'],
                    initial_data=○ ☆ ◇ □ ▷)
                    inbox=[],
                    outbox=[]
                    ,
 'B': Node(id='B', network=chain, 
                    neighbors=['A', 'C'],
                    initial_data=◇ ▷ ◁ □ ○)
                    inbox=[],
                    outbox=[]
                    ,
 'C': Node(id='C', network=chain, 
                    neighbors=['B', 'D'],
                    initial_data=◁ ▷ ☆ □ ◇)
                    inbox=[],
                    outbox=[]
                    ,
 'D': Node(id='D', network=chain, 
                    neighbors=['C', 'E'],
                    initial_data=○ ◁ □ ◇ ☆)
                    inbox=[],
                    outbox=[]
                    ,
 'E': Node(id='E', network=chain, 
                    neighbors=['D'],
                    initial_data=▷ ○ ◁ ☆ ◇)
                    inbox=[],
                    outbox=[]
                 

In [33]:
nodes['C'].take_turn()                                                     

```json
{
  "current_guess": null,
  "outgoing_messages": [
    { "to": "B", "content": "My symbols: ◁ ▷ ☆ □ ◇" },
    { "to": "D", "content": "My symbols: ◁ ▷ ☆ □ ◇" }
  ],
  "notes": "No messages received yet. Shared my full symbol set with neighbors B and D to gather initial data for comparison."
}
```


In [34]:
nodes['C'].send_message(nodes['B'],  "My symbols: ◁ ▷ ☆ □ ◇")
nodes['C'].send_message(nodes['D'],  "My symbols: ◁ ▷ ☆ □ ◇")

"Message 'My symbols...' sent to Node D'"

In [35]:
nodes

{'A': Node(id='A', network=chain, 
                    neighbors=['B'],
                    initial_data=○ ☆ ◇ □ ▷)
                    inbox=[],
                    outbox=[]
                    ,
 'B': Node(id='B', network=chain, 
                    neighbors=['A', 'C'],
                    initial_data=◇ ▷ ◁ □ ○)
                    inbox=[{'from': 'C', 'message': 'My symbols: ◁ ▷ ☆ □ ◇'}],
                    outbox=[]
                    ,
 'C': Node(id='C', network=chain, 
                    neighbors=['B', 'D'],
                    initial_data=◁ ▷ ☆ □ ◇)
                    inbox=[],
                    outbox=[{'to': 'B', 'message': 'My symbols: ◁ ▷ ☆ □ ◇'}, {'to': 'D', 'message': 'My symbols: ◁ ▷ ☆ □ ◇'}]
                    ,
 'D': Node(id='D', network=chain, 
                    neighbors=['C', 'E'],
                    initial_data=○ ◁ □ ◇ ☆)
                    inbox=[{'from': 'C', 'message': 'My symbols: ◁ ▷ ☆ □ ◇'}],
                    outbox=[]
                    ,

In [36]:
nodes['B'].take_turn()

```json
{
  "current_guess": null,
  "outgoing_messages": [
    {
      "to": "A",
      "content": "My symbols: ◇ ▷ ◁ □ ○"
    },
    {
      "to": "C",
      "content": "Thanks. Based on our cards, shared symbols: ◇ ▷ ◁ □"
    }
  ],
  "notes": "My card: ◇ ▷ ◁ □ ○ | C's card: ◁ ▷ ☆ □ ◇ | Common symbols between me and C: ◇ ▷ ◁ □ | No current unique candidate for common symbol among all cards yet."
}
```


In [37]:
nodes['B'].send_message(nodes['A'],  "My symbols: ◇ ▷ ◁ □ ○")
nodes['B'].send_message(nodes['C'],  "Thanks. Based on our cards, shared symbols: ◇ ▷ ◁ □")

"Message 'Thanks. Ba...' sent to Node C'"

In [38]:
nodes['C'].take_turn()

```json
{
  "current_guess": null,
  "outgoing_messages": [
    {
      "to": "B",
      "content": "My card: ◁ ▷ ☆ □ ◇"
    },
    {
      "to": "D",
      "content": "My card: ◁ ▷ ☆ □ ◇"
    }
  ],
  "notes": "My card is ◁ ▷ ☆ □ ◇. From B, I learned that B and I share ◇ ▷ ◁ □. That implies B's card includes these 4 symbols, possibly one other. I shared my card with both B and D to gather more information."
}
```


In [39]:
nodes['C'].send_message(nodes['B'],  "My card: ◁ ▷ ☆ □ ◇")
nodes['C'].send_message(nodes['D'],  "My card: ◁ ▷ ☆ □ ◇")

"Message 'My card: ◁...' sent to Node D'"

In [40]:
nodes['D'].take_turn()

```json
{
  "current_guess": null,
  "outgoing_messages": [
    {
      "to": "C",
      "content": "My symbols: ○ ◁ □ ◇ ☆"
    },
    {
      "to": "E",
      "content": "My symbols: ○ ◁ □ ◇ ☆"
    }
  ],
  "notes": "Shared my card with C and E. C's card is ◁ ▷ ☆ □ ◇. Current intersection with C is ◁, ☆, □, ◇. Still waiting for more cards to narrow down the common symbol."
}
```


In [41]:
nodes['D'].send_message(nodes['C'],  "My symbols: ○ ◁ □ ◇ ☆")
nodes['D'].send_message(nodes['E'],  "My symbols: ○ ◁ □ ◇ ☆")

"Message 'My symbols...' sent to Node E'"

In [42]:
nodes['E'].take_turn()

```json
{
  "current_guess": null,
  "outgoing_messages": [
    {
      "to": "D",
      "content": "My symbols: ▷ ○ ◁ ☆ ◇"
    }
  ],
  "notes": "I have compared my symbols ▷ ○ ◁ ☆ ◇ with D's symbols ○ ◁ □ ◇ ☆. The overlapping symbols are ○, ◁, ☆, and ◇. I’ve sent my full symbol list to D to help narrow down candidates further."
}
```


In [43]:
nodes['E'].send_message(nodes['D'],  "My symbols: ▷ ○ ◁ ☆ ◇")

"Message 'My symbols...' sent to Node D'"

In [44]:
nodes['D'].take_turn()

```json
{
  "current_guess": "☆",
  "outgoing_messages": [
    {
      "to": "C",
      "content": "Current candidate: ☆ ◁ □ ◇"
    },
    {
      "to": "E",
      "content": "Current candidate: ☆ ◁ ◇"
    }
  ],
  "notes": "Initial symbols: ○ ◁ □ ◇ ☆. Learned from C: ◁ ▷ ☆ □ ◇. Learned from E: ▷ ○ ◁ ☆ ◇. Intersecting all three sets gives: ☆ ◁ ◇. Then intersect with my card: ☆ ◁ ◇ ∩ ○ ◁ □ ◇ ☆ = ☆ ◁ ◇. Still ambiguous, but ☆ appears on all three known sets."
}
```


In [45]:
nodes['D'].send_message(nodes['C'],  "Current candidate: ☆ ◁ □ ◇")
nodes['D'].send_message(nodes['E'],  "Current candidate: ☆ ◁ ◇")

"Message 'Current ca...' sent to Node E'"

In [46]:
nodes['C'].take_turn()

```json
{
  "current_guess": "◇",
  "outgoing_messages": [
    {
      "to": "B",
      "content": "D's card is: ○ ◁ □ ◇ ☆"
    },
    {
      "to": "B",
      "content": "D's current candidate: ☆ ◁ □ ◇"
    }
  ],
  "notes": "My card: ◁ ▷ ☆ □ ◇. B says shared symbols with me: ◇ ▷ ◁ □, so B must also have those. D's symbols: ○ ◁ □ ◇ ☆. This confirms ◇, ◁, □, and ☆ are on at least 3 cards (mine, D's, and B's). Only ◇ appears on all known sets from me, B, and D. Therefore, ◇ is the best candidate for the common symbol."
}
```


In [47]:
common_symbol

'◇'

In [48]:
nodes['C'].send_message(nodes['B'],  "D's card is: ○ ◁ □ ◇ ☆")
nodes['C'].send_message(nodes['B'],  "D's current candidate: ☆ ◁ □ ◇")

"Message 'D's curren...' sent to Node B'"

In [49]:
nodes['B'].take_turn()

{
  "current_guess": null,
  "outgoing_messages": [
    {
      "to": "A",
      "content": "My symbols: ◇ ▷ ◁ □ ○"
    },
    {
      "to": "C",
      "content": "Thanks. Based on my symbols and yours, current candidates are: ◇ ▷ ◁ □"
    }
  ],
  "notes": "My card: ◇ ▷ ◁ □ ○. C's card: ◁ ▷ ☆ □ ◇. D’s card (from C): ○ ◁ □ ◇ ☆. Intersecting all known cards (mine, C, D): ◁ □ ◇. Tracking these as current candidates."
}


In [50]:
nodes['B'].send_message(nodes['A'],  "My symbols: ◇ ▷ ◁ □ ○")
nodes['B'].send_message(nodes['C'],  "Thanks. Based on my symbols and yours, current candidates are: ◇ ▷ ◁ □")

"Message 'Thanks. Ba...' sent to Node C'"

In [51]:
nodes['B'].take_turn()

{
  "current_guess": "◇",
  "outgoing_messages": [
    {
      "to": "A",
      "content": "My card: ◇ ▷ ◁ □ ○"
    },
    {
      "to": "C",
      "content": "Thanks. My card: ◇ ▷ ◁ □ ○"
    }
  ],
  "notes": "I have ◇ ▷ ◁ □ ○. C has ◁ ▷ ☆ □ ◇. D has ○ ◁ □ ◇ ☆. Common symbols on all 3 seen so far (me, C, and D) are ◇, ◁, and □. But only ◇ appears on all three—C, D (via C), and mine. Thus, ◇ is the only candidate that appears on all 3 known cards. Set current_guess to ◇."
}


In [52]:
nodes['B'].send_message(nodes['A'],  "My card: ◇ ▷ ◁ □ ○")
nodes['B'].send_message(nodes['C'],  "Thanks. My card: ◇ ▷ ◁ □ ○")

"Message 'Thanks. My...' sent to Node C'"

In [53]:
nodes['C'].take_turn()

```json
{
  "current_guess": "◇",
  "outgoing_messages": [
    {
      "to": "B",
      "content": "Considering all data from your card and D's messages, the shared symbol among all is likely ◇."
    },
    {
      "to": "D",
      "content": "From your card, my card, and B's, ◇ appears on all. It may be the common symbol."
    }
  ],
  "notes": "My symbols: ◁ ▷ ☆ □ ◇. B's card: ◇ ▷ ◁ □ ○. D's card: ○ ◁ □ ◇ ☆. Intersection of B, D, and me is ◇, ◁, □. But ◁ is not on ☆, and □ is not on all 5. ☆ is missing from B. So ◇ is the only symbol that appears on all three cards I know about. ◇ is the best candidate for the common symbol."
}
```


In [54]:
nodes['C'].send_message(nodes['B'],  "Considering all data from your card and D's messages, the shared symbol among all is likely ◇.")
nodes['C'].send_message(nodes['D'],  "From your card, my card, and B's, ◇ appears on all. It may be the common symbol.")

"Message 'From your ...' sent to Node D'"

In [55]:
nodes['D'].take_turn()

```json
{
  "current_guess": "◇",
  "outgoing_messages": [
    {
      "to": "C",
      "content": "My card: ○ ◁ □ ◇ ☆. I agree ◇ is on all of our cards. Do you know A or D's card?"
    },
    {
      "to": "E",
      "content": "My card: ○ ◁ □ ◇ ☆. ◇ is on mine, yours, and C's. Any info from A or D?"
    }
  ],
  "notes": "I know my card: ○ ◁ □ ◇ ☆. C's card: ◁ ▷ ☆ □ ◇. E’s card: ▷ ○ ◁ ☆ ◇. From this, ◇ is indeed the only symbol common across all three so far. If C is correct that B’s card also has ◇, we just need to hear from A and D to confirm."
}
```


In [56]:
nodes['D'].send_message(nodes['C'],  "My card: ○ ◁ □ ◇ ☆. I agree ◇ is on all of our cards. Do you know A or D's card?")
nodes['D'].send_message(nodes['E'],  "My card: ○ ◁ □ ◇ ☆. ◇ is on mine, yours, and C's. Any info from A or D?")

"Message 'My card: ○...' sent to Node E'"

In [57]:
nodes['C'].take_turn()

{
  "current_guess": "◇",
  "outgoing_messages": [
    {
      "to": "B",
      "content": "D's card: ○ ◁ □ ◇ ☆. From my card and yours, common symbols: ◇ ▷ ◁ □. D confirms ◇ is on all cards. I agree."
    },
    {
      "to": "D",
      "content": "B's card: ◇ ▷ ◁ □ ○. From my card and B's, common symbols: ◇ ▷ ◁ □. Agreed, ◇ is on all our cards."
    }
  ],
  "notes": "All three cards (mine, B's, and D's) contain the symbol ◇. The intersection of known cards is only ◇, so it's the only symbol known to appear on all three cards. Set current_guess to ◇."
}


In [58]:
nodes['C'].send_message(nodes['B'],  "D's card: ○ ◁ □ ◇ ☆. From my card and yours, common symbols: ◇ ▷ ◁ □. D confirms ◇ is on all cards. I agree.")
nodes['C'].send_message(nodes['D'],  "B's card: ◇ ▷ ◁ □ ○. From my card and B's, common symbols: ◇ ▷ ◁ □. Agreed, ◇ is on all our cards.")

"Message 'B's card: ...' sent to Node D'"

In [59]:
nodes['E'].take_turn()

```json
{
  "current_guess": "◇",
  "outgoing_messages": [],
  "notes": "My card: ▷ ○ ◁ ☆ ◇. D's card: ○ ◁ □ ◇ ☆. The intersection of our cards is: ○ ◁ ◇ ☆. D said ◇ is on mine, yours, and C's. So ◇ appears on at least 3 cards. Since I cannot message A or C directly and no other new info is accessible, and ◇ is the only known symbol common to my card and all reported positions (including C), I conclude ◇ is the common symbol."
}
```


In [60]:
nodes

{'A': Node(id='A', network=chain, 
                    neighbors=['B'],
                    initial_data=○ ☆ ◇ □ ▷)
                    inbox=[{'from': 'B', 'message': 'My symbols: ◇ ▷ ◁ □ ○'}, {'from': 'B', 'message': 'My symbols: ◇ ▷ ◁ □ ○'}, {'from': 'B', 'message': 'My card: ◇ ▷ ◁ □ ○'}],
                    outbox=[]
                    ,
 'B': Node(id='B', network=chain, 
                    neighbors=['A', 'C'],
                    initial_data=◇ ▷ ◁ □ ○)
                    inbox=[{'from': 'C', 'message': 'My symbols: ◁ ▷ ☆ □ ◇'}, {'from': 'C', 'message': 'My card: ◁ ▷ ☆ □ ◇'}, {'from': 'C', 'message': "D's card is: ○ ◁ □ ◇ ☆"}, {'from': 'C', 'message': "D's current candidate: ☆ ◁ □ ◇"}, {'from': 'C', 'message': "Considering all data from your card and D's messages, the shared symbol among all is likely ◇."}, {'from': 'C', 'message': "D's card: ○ ◁ □ ◇ ☆. From my card and yours, common symbols: ◇ ▷ ◁ □. D confirms ◇ is on all cards. I agree."}],
                    outbox=[{'to

In [61]:
nodes['A'].take_turn()

```json
{
  "current_guess": null,
  "outgoing_messages": [],
  "notes": "My card: ○ ☆ ◇ □ ▷. B's card: ◇ ▷ ◁ □ ○. The overlap between my card and B's is: ◇ ▷ □ ○. Still need information from the other 3 nodes to determine the single common symbol. Cannot proceed further since I can only communicate with B and have no new messages to send or receive at this point."
}
```


In [62]:
nodes['A'].take_turn()

```json
{
  "current_guess": null,
  "outgoing_messages": [],
  "notes": "My symbols are: ○ ☆ ◇ □ ▷. B's symbols are: ◇ ▷ ◁ □ ○. The intersection between our cards is: ◇ □ ▷ ○. That is our current candidate set. Since I can only talk to B, and already got B's full card, I will wait to receive additional information from B for now."
}
```


In [63]:
nodes['A'].take_turn()

```json
{
  "current_guess": null,
  "outgoing_messages": [],
  "notes": "My symbols are: ○ ☆ ◇ □ ▷. B's symbols are: ◇ ▷ ◁ □ ○. Intersecting B's and my symbols gives: ◇ ▷ □ ○. Still too many candidates to guess uniquely. Waiting for more data from B, cannot contact others directly."
}
```


In [64]:
nodes['B'].take_turn()

{
  "current_guess": "◇",
  "outgoing_messages": [
    {
      "to": "A",
      "content": "My card: ◇ ▷ ◁ □ ○. From C I learned: C's card is ◁ ▷ ☆ □ ◇, D's card is ○ ◁ □ ◇ ☆. Common symbols on all 3 of our cards: ◇ ◁ □. C and D believe ◇ is on all cards. Current guess: ◇. Your symbols?"
    },
    {
      "to": "C",
      "content": "Agreed with your conclusion. My current guess is ◇."
    }
  ],
  "notes": "My card: ◇ ▷ ◁ □ ○. C's card: ◁ ▷ ☆ □ ◇. D's card: ○ ◁ □ ◇ ☆. Common symbols among my, C's, and D's cards: ◇ ◁ □. C and D favor ◇ as the only one definitely shared. Awaiting A's data to confirm further."
}


In [65]:
nodes['B'].send_message(nodes['A'],  "My card: ◇ ▷ ◁ □ ○. From C I learned: C's card is ◁ ▷ ☆ □ ◇, D's card is ○ ◁ □ ◇ ☆. Common symbols on all 3 of our cards: ◇ ◁ □. C and D believe ◇ is on all cards. Current guess: ◇. Your symbols?")
nodes['B'].send_message(nodes['C'],  "Agreed with your conclusion. My current guess is ◇.")

"Message 'Agreed wit...' sent to Node C'"

In [66]:
nodes['A'].take_turn()

```json
{
  "current_guess": "◇",
  "outgoing_messages": [
    {
      "to": "B",
      "content": "My symbols: ○ ☆ ◇ □ ▷. I also see ◇ on all cards I've heard so far. Agree with guess ◇."
    }
  ],
  "notes": "I compared my card (○ ☆ ◇ □ ▷) with the cards from B (◇ ▷ ◁ □ ○), C (◁ ▷ ☆ □ ◇), and D (○ ◁ □ ◇ ☆) as reported by B. Common symbols across all cards are □ and ◇. But only ◇ appears in all 5 (mine, B, C, D), because ☆ is missing from B, ▷ from D, etc. So ◇ is the only universally shared one so far."
}
```


In [69]:
nodes['A'].send_message(nodes['B'],"My symbols: ○ ☆ ◇ □ ▷. I also see ◇ on all cards I've heard so far. Agree with guess ◇.")

"Message 'My symbols...' sent to Node B'"